In [1]:
# Helper functions for evaluation
def yolo_to_sv_detections(results, frame_width, frame_height):
    """Convert YOLO results to supervision Detections format"""
    boxes = results.boxes.xyxy.cpu().numpy()
    scores = results.boxes.conf.cpu().numpy()
    class_ids = results.boxes.cls.cpu().numpy().astype(int)
    
    return sv.Detections(
        xyxy=boxes,
        confidence=scores,
        class_id=class_ids
    )

def read_yolo_label(label_path, frame_width, frame_height):
    """Read YOLO format label file and convert to supervision Detections"""
    boxes = []
    class_ids = []
    
    with open(label_path, 'r') as f:
        for line in f.readlines():
            parts = line.strip().split()
            class_id, x_center, y_center, width, height = map(float, parts)
            
            # Convert normalized coordinates to pixel coordinates
            x1 = (x_center - width/2) * frame_width
            y1 = (y_center - height/2) * frame_height
            x2 = (x_center + width/2) * frame_width
            y2 = (y_center + height/2) * frame_height
            
            boxes.append([x1, y1, x2, y2])
            class_ids.append(int(class_id))
    
    if not boxes:
        return sv.Detections.empty()
    
    return sv.Detections(
        xyxy=np.array(boxes),
        class_id=np.array(class_ids),
        confidence=np.ones(len(class_ids))  # Ground truth has confidence 1.0
    )

# Function to get image dimensions from TIFF
def get_tiff_dimensions(image_path):
    """Get dimensions from a TIFF file using rasterio"""
    with rasterio.open(image_path) as src:
        height = src.height
        width = src.width
    return height, width

In [ ]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
from ultralytics import YOLO
from PIL import Image

def visualize_matching(img_path, gt_detections, pred_detections, matched_gt, output_path):
    """Visualize ground truth, predictions, and matches for debugging"""
    with rasterio.open(img_path) as src:
        image_data = src.read()
        if image_data.shape[0] >= 3:  
            image = np.transpose(image_data[:3], (1, 2, 0))
            if image.max() > 255:
                image = (image / 65535 * 255).astype(np.uint8)
            elif image.max() <= 1:
                image = (image * 255).astype(np.uint8)
        else:  
            image = np.stack([image_data[0]] * 3, axis=2)
            if image.max() > 255:
                image = (image / 65535 * 255).astype(np.uint8)
            elif image.max() <= 1:
                image = (image * 255).astype(np.uint8)
    
    plt.figure(figsize=(12, 12))
    plt.imshow(image)
    
    # Draw ground truths in different colors based on match status
    for i, gt in enumerate(gt_detections):
        x1, y1, x2, y2 = gt[:4]
        if matched_gt[i]:
            color = 'lime'  # Matched ground truths
        else:
            color = 'yellow'  # Unmatched ground truths (false negatives)
        plt.gca().add_patch(plt.Rectangle((x1, y1), x2-x1, y2-y1, fill=False, edgecolor=color, linewidth=2))
    
    # Draw predictions in red (false positives will be the only red ones)
    for pred in pred_detections:
        x1, y1, x2, y2 = pred[:4]
        plt.gca().add_patch(plt.Rectangle((x1, y1), x2-x1, y2-y1, fill=False, edgecolor='red', linewidth=2))
    
    plt.title(f'Match Visualization: Green=TP, Yellow=FN, Red=FP')
    plt.axis('off')
    plt.savefig(output_path)
    plt.close()


# Define helper functions with debugging
def get_tiff_dimensions(img_path):
    """Get dimensions of a TIFF image."""
    try:
        with Image.open(img_path) as img:
            width, height = img.size
        return height, width
    except Exception as e:
        print(f"Error opening image {img_path}: {e}")
        # Return default dimensions if there's an error
        return 640, 640

def yolo_to_sv_detections(results, width, height):
    """Convert YOLO results to a standard format for evaluation."""
    detections = []
    
    # Debug: Check what's in the results
    try:
        # For YOLOv8 results, extract boxes if they exist
        if hasattr(results, 'boxes') and len(results.boxes) > 0:
            boxes = results.boxes.xyxy.cpu().numpy()  # Get the boxes in x1,y1,x2,y2 format
            confs = results.boxes.conf.cpu().numpy()  # Get confidence scores
            cls_ids = results.boxes.cls.cpu().numpy().astype(int)  # Get class IDs
            
            print(f"Found {len(boxes)} detections with classes: {np.unique(cls_ids)}")
            
            for box, conf, cls_id in zip(boxes, confs, cls_ids):
                x1, y1, x2, y2 = box
                detections.append([x1, y1, x2, y2, cls_id, conf])
        else:
            print("No boxes found in results or results has no 'boxes' attribute")
            # Alternative parsing for different YOLO versions
            if hasattr(results, 'pred') and len(results.pred) > 0:
                for pred in results.pred[0]:
                    if len(pred) >= 6:  # Make sure we have x1,y1,x2,y2,conf,cls
                        x1, y1, x2, y2, conf, cls_id = pred.cpu().numpy().tolist()
                        detections.append([x1, y1, x2, y2, int(cls_id), conf])
    except Exception as e:
        print(f"Error parsing YOLO results: {e}")
        print(f"Results type: {type(results)}")
        
    return detections

def read_yolo_label(label_path, width, height):
    """Read YOLO format labels and convert to the same format as detections."""
    detections = []
    
    try:
        if not os.path.exists(label_path):
            print(f"Warning: Label file does not exist: {label_path}")
            return detections
            
        with open(label_path, 'r') as f:
            lines = f.readlines()
            
        print(f"Found {len(lines)} ground truth objects in {label_path}")
        
        for line in lines:
            parts = line.strip().split()
            if len(parts) >= 5:  # At least class_id and 4 bbox coords
                cls_id = int(parts[0])
                # YOLO format is [cls_id, x_center, y_center, width, height]
                # Convert normalized coordinates to pixel values
                x_center = float(parts[1]) * width
                y_center = float(parts[2]) * height
                w = float(parts[3]) * width
                h = float(parts[4]) * height
                
                # Convert to [x1, y1, x2, y2, cls_id, 1.0] format
                # where 1.0 is a placeholder confidence score for ground truth
                x1 = x_center - w/2
                y1 = y_center - h/2
                x2 = x_center + w/2
                y2 = y_center + h/2
                
                detections.append([x1, y1, x2, y2, cls_id, 1.0])
    except Exception as e:
        print(f"Error reading label file {label_path}: {e}")
    
    return detections

def get_test_samples(base_dir, test_dir_pattern='**/dataset/images/test'):
    """Find all test samples by searching for the test directory pattern."""
    import glob
    
    test_dirs = glob.glob(os.path.join(base_dir, test_dir_pattern), recursive=True)
    
    if not test_dirs:
        print(f"Warning: No test directories found matching pattern '{test_dir_pattern}' in {base_dir}")
        return []
    
    print(f"Found {len(test_dirs)} test directories: {test_dirs}")
    
    pairs = []
    for test_dir in test_dirs:
        img_dir = test_dir
        # Try to locate corresponding labels directory
        label_dir = test_dir.replace('images', 'labels')
        if not os.path.exists(label_dir):
            label_dir = os.path.join(os.path.dirname(test_dir), 'labels')
        
        print(f"Checking for images in {img_dir}")
        print(f"Checking for labels in {label_dir}")
        
        # Find image files
        img_files = []
        for ext in ['.jpg', '.jpeg', '.png', '.tif', '.tiff']:
            img_files.extend(glob.glob(os.path.join(img_dir, f'*{ext}')))
        
        print(f"Found {len(img_files)} image files")
        
        for img_path in img_files:
            base_name = os.path.splitext(os.path.basename(img_path))[0]
            label_path = os.path.join(label_dir, f"{base_name}.txt")
            
            if os.path.exists(label_path):
                pairs.append((img_path, label_path))
                
    print(f"Found {len(pairs)} valid image-label pairs")
    return pairs

# Define the class mapping - MODIFY THIS TO MATCH YOUR DATASET
# Pretrained YOLOv8 uses COCO classes by default, solar panels would be a custom class
class_map = {
    0: "solar_panel",  # If you have a custom model with class 0 as solar panel
    # Add other classes if needed
}

# Show COCO class list (for reference if using pretrained model)
coco_classes = ["person", "bicycle", "car", "motorcycle", "airplane", "bus", "train", "truck", "boat", 
                "traffic light", "fire hydrant", "stop sign", "parking meter", "bench", "bird", "cat", 
                "dog", "horse", "sheep", "cow", "elephant", "bear", "zebra", "giraffe", "backpack", 
                "umbrella", "handbag", "tie", "suitcase", "frisbee", "skis", "snowboard", "sports ball", 
                "kite", "baseball bat", "baseball glove", "skateboard", "surfboard", "tennis racket", 
                "bottle", "wine glass", "cup", "fork", "knife", "spoon", "bowl", "banana", "apple", 
                "sandwich", "orange", "broccoli", "carrot", "hot dog", "pizza", "donut", "cake", "chair", 
                "couch", "potted plant", "bed", "dining table", "toilet", "tv", "laptop", "mouse", "remote", 
                "keyboard", "cell phone", "microwave", "oven", "toaster", "sink", "refrigerator", "book", 
                "clock", "vase", "scissors", "teddy bear", "hair drier", "toothbrush"]
print(f"COCO classes (if using pretrained model): {coco_classes}")

# Define base directory
base_dir = os.getcwd()
print(f"Current working directory: {base_dir}")

# Load the model
best_model_path = 'YOLOv8n.pt'  # This is a pretrained model
print(f"Loading model from {best_model_path}")

if not os.path.exists(best_model_path):
    print(f"Warning: Model file {best_model_path} not found, trying to download if it's a standard model")

try:
    best_model = YOLO(best_model_path)
    print(f"Model loaded successfully")
except Exception as e:
    print(f"Error loading model: {e}")
    # Try to use a standard model name
    try:
        print("Trying to load from standard name 'yolov8n'")
        best_model = YOLO('yolov8n')
        print("Successfully loaded yolov8n model")
    except Exception as e:
        print(f"Error loading standard model: {e}")
        raise

# Get test samples using flexible approach
random_test_samples = get_test_samples(base_dir)

if not random_test_samples:
    print("No test samples found. Using hardcoded paths from error log for testing:")
    # Extract paths from error log and create sample pairs
    base_image_dir = r"C:\Users\HP\Downloads\labels-20250212T103318Z-001\labels\solar_panel_project\dataset\images\test"
    sample_images = [
        "solarpanels_native_2__x0_81_y0_11582_dxdy_416.tif",
        "solarpanels_native_2__x0_11803_y0_357_dxdy_416.tif",
        "solarpanels_native_2__x0_9770_y0_7171_dxdy_416.tif",
        "solarpanels_native_3__x0_10149_y0_9741_dxdy_416.tif"
    ]
    
    random_test_samples = []
    for img_name in sample_images:
        img_path = os.path.join(base_image_dir, img_name)
        # Try to find corresponding label file
        label_base = os.path.splitext(img_name)[0]
        label_path = os.path.join(base_image_dir.replace('images', 'labels'), f"{label_base}.txt")
        random_test_samples.append((img_path, label_path))
    
    print(f"Created {len(random_test_samples)} sample pairs from error log")

# Limit samples for testing if needed
if len(random_test_samples) > 10:
    import random
    random.shuffle(random_test_samples)
    random_test_samples = random_test_samples[:10]
    print(f"Limited to 10 random samples for testing")

# Define target class for solar panels
# IMPORTANT: Modify this to match your specific class ID
TARGET_CLASS = 0  # Assuming 0 is solar panel in your custom model
# If using pretrained YOLOv8 with COCO classes, you might need to use a different ID
# or adapt the code to match semantically similar classes

# Define IoU and confidence thresholds
iou_thresholds = [0.1, 0.3, 0.5, 0.7, 0.9]
conf_thresholds = [0.1, 0.3, 0.5, 0.7, 0.9]

# Initialize results dictionaries
results_dict = {
    'precision': np.zeros((len(iou_thresholds), len(conf_thresholds))),
    'recall': np.zeros((len(iou_thresholds), len(conf_thresholds))),
    'f1': np.zeros((len(iou_thresholds), len(conf_thresholds)))
}

# Process each IoU and confidence threshold
for iou_idx, iou_threshold in enumerate(iou_thresholds):
    for conf_idx, conf_threshold in enumerate(conf_thresholds):
        print(f"\n===== Processing: IoU={iou_threshold}, Conf={conf_threshold} =====")
        
        # Using a different approach for binary classification: track TP, FP, TN, FN
        tp = 0  # True positives
        fp = 0  # False positives
        fn = 0  # False negatives
        
        # Debug counters
        total_gt_objects = 0
        total_pred_objects = 0
        
        # Process each test image
        for img_idx, (img_path, label_path) in enumerate(random_test_samples):
            try:
                print(f"\nImage {img_idx+1}/{len(random_test_samples)}: {os.path.basename(img_path)}")
                
                # Check if files exist
                if not os.path.exists(img_path):
                    print(f"Warning: Image file does not exist: {img_path}")
                    continue
                    
                # Get image dimensions
                height, width = get_tiff_dimensions(img_path)
                print(f"Image dimensions: {width}x{height}")
                
                # Get ground truth labels
                gt_detections = read_yolo_label(label_path, width, height)
                total_gt_objects += len(gt_detections)
                
                # Get predictions
                print(f"Running inference with conf={conf_threshold}, iou={iou_threshold}")
                results = best_model(img_path, conf=conf_threshold, iou=iou_threshold)
                pred_detections = yolo_to_sv_detections(results[0], width, height)
                total_pred_objects += len(pred_detections)
                
                # Filter for target class if specified
                if TARGET_CLASS is not None:
                    gt_detections = [det for det in gt_detections if det[4] == TARGET_CLASS]
                    pred_detections = [det for det in pred_detections if det[4] == TARGET_CLASS]
                    print(f"After filtering for class {TARGET_CLASS}: {len(gt_detections)} GT, {len(pred_detections)} predictions")
                
                # Calculate matches between predictions and ground truth using IoU
                gt_matched = [False] * len(gt_detections)
                
                for pred in pred_detections:
                    matched = False
                    pred_cls = pred[4]
                    pred_conf = pred[5]
                    
                    for i, gt in enumerate(gt_detections):
                        if not gt_matched[i] and gt[4] == pred_cls:
                            # Calculate IoU between boxes
                            pred_box = pred[:4]  # [x1, y1, x2, y2]
                            gt_box = gt[:4]      # [x1, y1, x2, y2]
                            
                            # Calculate intersection
                            x_left = max(pred_box[0], gt_box[0])
                            y_top = max(pred_box[1], gt_box[1])
                            x_right = min(pred_box[2], gt_box[2])
                            y_bottom = min(pred_box[3], gt_box[3])
                            
                            if x_right < x_left or y_bottom < y_top:
                                intersection_area = 0
                            else:
                                intersection_area = (x_right - x_left) * (y_bottom - y_top)
                            
                            # Calculate union
                            pred_area = (pred_box[2] - pred_box[0]) * (pred_box[3] - pred_box[1])
                            gt_area = (gt_box[2] - gt_box[0]) * (gt_box[3] - gt_box[1])
                            union_area = pred_area + gt_area - intersection_area
                            
                            iou = intersection_area / union_area if union_area > 0 else 0
                            
                            if iou > 0.3:  # Debug any reasonable IoU
                                print(f"DEBUG - IoU: {iou:.4f} between:")
                                print(f"  PRED: {pred_box}")
                                print(f"  GT: {gt_box}")
                            
                            if iou > iou_threshold:
                                matched = True
                                gt_matched[i] = True
                                p += 1
                    
                    if not matched:
                        fp += 1
                        print(f"False positive: class={pred_cls}, conf={pred_conf:.2f}")
                
                # Count false negatives (GT objects that weren't matched)
                current_fn = sum(1 for matched in gt_matched if not matched)
                fn += current_fn
                if current_fn > 0:
                    print(f"False negatives: {current_fn}")
                
            except Exception as e:
                print(f"Error processing {img_path}: {e}")
                import traceback
                traceback.print_exc()
        
        print(f"\nSummary for IoU={iou_threshold}, Conf={conf_threshold}:")
        print(f"Total GT objects: {total_gt_objects}")
        print(f"Total predicted objects: {total_pred_objects}")
        print(f"True positives: {tp}")
        print(f"False positives: {fp}")
        print(f"False negatives: {fn}")
        
        # Calculate metrics
        precision = tp / (tp + fp) if (tp + fp) > 0 else 0
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0
        f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
        
        # Store results
        results_dict['precision'][iou_idx, conf_idx] = precision
        results_dict['recall'][iou_idx, conf_idx] = recall
        results_dict['f1'][iou_idx, conf_idx] = f1
        
        print(f"Metrics - Precision: {precision:.3f}, Recall: {recall:.3f}, F1: {f1:.3f}")

# Generate tables and visualizations
metrics = ['Precision', 'Recall', 'F1-Score']
for metric in metrics:
    metric_lower = metric.lower().replace('-', '')
    if metric_lower == 'f1score':
        metric_lower = 'f1'
    
    print(f"\n{metric} Table (rows: IoU thresholds, columns: Confidence thresholds)")
    df = pd.DataFrame(
        results_dict[metric_lower],
        index=[f'IoU={iou:.1f}' for iou in iou_thresholds],
        columns=[f'Conf={conf:.1f}' for conf in conf_thresholds]
    )
    print(df.round(3))
    
    # Save table
    df.to_csv(os.path.join(base_dir, f'{metric_lower}_table.csv'))
    
    # Create heatmap
    fig, ax = plt.subplots(figsize=(10, 8))
    sns.heatmap(df, annot=True, cmap="YlGnBu", fmt=".3f", vmin=0, vmax=1, ax=ax)
    ax.set_title(f"{metric} at Different IoU and Confidence Thresholds")
    plt.tight_layout()
    plt.savefig(os.path.join(base_dir, f'{metric_lower}_heatmap.png'))
    plt.show()

COCO classes (if using pretrained model): ['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush']
Current working directory: C:\Users\HP\Downloads\labels-20250212T103318Z-001\labels
Loading mode